<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [1]:
!wget  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

--2022-07-11 15:38:54--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261483817 (249M) [application/zip]
Saving to: ‘concrete_data_week4.zip’

concrete_data_week4 100%[===================>] 249.37M  17.8MB/s    in 16s     

2022-07-11 15:39:11 (16.0 MB/s) - ‘concrete_data_week4.zip’ saved [261483817/261483817]



In [2]:
!unzip -q concrete_data_week4.zip

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [3]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.applications import VGG16
import matplotlib.pyplot as plt
import tensorflow.keras as keras
import numpy as np
import os

datadir = '/content/concrete_data_week4/'

In [19]:
datagen = ImageDataGenerator(preprocessing_function = preprocess_input)
train_gen = datagen.flow_from_directory(
    datadir+"train/",
    target_size=(224,224),
    batch_size=100,
    class_mode="categorical")

val_gen = datagen.flow_from_directory(
    datadir+"valid/",
    target_size=(224,224),
    batch_size=100,
    class_mode="categorical")

test_gen = datagen.flow_from_directory(
    datadir+"test/",
    target_size=(224,224),
    batch_size=100,
    class_mode="categorical")


Found 30001 images belonging to 2 classes.
Found 9501 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


In [6]:
model = Sequential()

model.add(VGG16(include_top=False, pooling="max", weights="imagenet"))
model.add(Dense(2, activation="softmax"))
model.layers[0].trainable = False
model.summary()

58900480/58889256 [==============================] - 2s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 dense (Dense)               (None, 2)                 1026      
                                                                 
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [12]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['accuracy'])

num_epochs = 2
per_train = len(train_gen)
per_valid = len(val_gen)


In [13]:
history = model.fit(
    train_gen,
    steps_per_epoch= per_train,
    epochs= num_epochs,
    validation_data= val_gen,
    validation_steps= per_valid,
    verbose=1,
)

Epoch 1/2
301/301 [==============================] - 215s 683ms/step - loss: 0.0442 - accuracy: 0.9923 - val_loss: 0.0208 - val_accuracy: 0.9957
Epoch 2/2
301/301 [==============================] - 215s 716ms/step - loss: 0.0190 - accuracy: 0.9971 - val_loss: 0.0269 - val_accuracy: 0.9956


In [14]:
model.save("vgg16_model.h5")

<a id="item43"></a>

## Part 2

Use the following cells to evaluate your models.

In [20]:
testing_generator = datagen.flow_from_directory(
    "concrete_data_week4/test", target_size=(224,224), shuffle=False)

performance_vgg16 = model.evaluate_generator(test_gen)
print("Performance of the VGG16-trained model")
print("Loss: {}".format(round(performance_vgg16[0], 5)))
print("Accuracy: {}".format(round(performance_vgg16[1], 5)))

Found 500 images belonging to 2 classes.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  after removing the cwd from sys.path.


Performance of the VGG16-trained model
Loss: 0.06003
Accuracy: 0.986


<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [24]:
predictions_vgg16 = model.predict_generator(test_gen, steps=1)


def print_prediction(prediction):
    if prediction[0] > prediction[1]:
        print("Negative ({}% certainty)".format(round(prediction[0] * 100, 1)))
    elif prediction[1] > prediction[0]:
        print("Positive ({}% certainty)".format(round(prediction[1] * 100, 1)))
    else:
        print("Unsure (prediction split 50–50)")


print("First five predictions for the VGG16-trained model:")
for i in range(5):
    print_prediction(predictions_vgg16[i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  """Entry point for launching an IPython kernel.


First five predictions for the VGG16-trained model:
Negative (100.0% certainty)
Negative (100.0% certainty)
Positive (100.0% certainty)
Negative (100.0% certainty)
Negative (100.0% certainty)


### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).